In [ ]:
from pyspark.sql import SparkSession

spark_session = SparkSession\
        .builder\
        .master("local") \
        .appName("Question_A4")\
        .getOrCreate()

spark_context = spark_session.sparkContext

In [ ]:
textEN = spark_context.textFile("/home/ubuntu/LDSA/lab2/europarl-v7.sv-en.en")
textSW = spark_context.textFile("/home/ubuntu/LDSA/lab2/europarl-v7.sv-en.sv")
pipedEN = textEN.pipe("/home/ubuntu/LDSA/lab2/tools/tokenizer.perl -l en")
pipedSW = textSW.pipe("/home/ubuntu/LDSA/lab2/tools/tokenizer.perl -l sv")

In [ ]:
indexedEN = pipedEN.zipWithIndex()

In [ ]:
indexedENInverted = indexedEN.map(lambda x: (x[1], x[0]))

In [ ]:
indexedSWInverted = pipedSW.zipWithIndex()\
.map(lambda x: (x[1], x[0]))

In [ ]:
joinedCorpae = indexedSWInverted.join(indexedENInverted)

In [ ]:
joinedCorpae.take(5)

In [ ]:
wordsSplit = joinedCorpae.map(lambda x: (x[0], (x[1][0].split(" "), x[1][1].split(" "))))

In [ ]:
wordsSplit.take(2)

In [ ]:
wordsSplit.count()

In [ ]:
wordsSplitFiltered = wordsSplit.filter(lambda element: not " " == element[1][1] or not " " == element[1][0])\
.filter(lambda element: len(element[1][1]) < 6)\
.filter(lambda element: len(element[1][0]) < 6)\
.filter(lambda element: len(element[1][0])==len(element[1][1]))

In [ ]:
wordsSplitFiltered.take(5)

In [ ]:
rdd1 = wordsSplitFiltered.flatMap(lambda x: x[1][0])
rdd2 = wordsSplitFiltered.flatMap(lambda x: x[1][1])
wordsJoined = rdd1.zip(rdd2)

In [ ]:
wordsJoined.count()

In [ ]:
wordsJoined.take(2)

In [ ]:
wordsJoined.map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)\
.sortByKey(True).take(10)

In [ ]:
wordsJoined.map(lambda word: (word, 1)) \
            .reduceByKey(lambda a, b: a + b)\
            .sortBy(lambda x: x[1], False).take(30)